In [1]:
%matplotlib inline 
import pandas as pd 

In [2]:
import os
os.chdir('C://Users//2918261//Dropbox//Corner Office Interviews')
# ('C://Users//Toshiba//Dropbox//Corner Office Interviews')
# C://Users//2918261//Dropbox//Corner Office Interviews         
# Toshiba//Dropbox//Corner Office Interviews')
# ('C://Users//2918261//Courses//Oslo Summer School//notebooks//Interviews//Text')
os.getcwd()

'C:\\Users\\2918261\\Dropbox\\Corner Office Interviews'

In [3]:
pd.set_option('display.max_colwidth', 120)

In [4]:
interviews = pd.read_json('C:/Users/2918261/Dropbox/Corner Office Interviews/ComparisonCorpusText/Json Tables for ML/interviews_full.json')

## Similar preprocessing from Topic Modelling

In [6]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

C:\Users\2918261\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\2918261\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


In [7]:
data = interviews.text.values.tolist()

In [8]:
data = [re.sub("\'", "", sent) for sent in data]

In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [10]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

C:\Users\2918261\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\2918261\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


## looking specifically for 3 grams

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
vectorizer = CountVectorizer(analyzer='word',
                             ngram_range = (3, 3),
                             min_df = 10,                      # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

In [22]:
data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [23]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  2.902957692229723 %


In [25]:
df= pd.DataFrame(data_dense,
                 columns=vectorizer.get_feature_names())

In [27]:
df.sum().sort_values(ascending=False)

spend lot time             133
want make sure              76
new york city               69
work really hard            50
make sure people            45
talk little bit             43
think really important      40
make good decision          36
ask lot question            33
need make sure              32
think lot people            31
make lot money              28
tell little bit             28
make people feel            27
new york time               27
make thing happen           27
people really want          26
work life balance           26
make sure know              26
want know want              24
try make sure               24
think important thing       24
know know know              23
work hour week              23
just little bit             23
high school college         23
make sure everybody         22
try new thing               22
lot different thing         22
chief operating officer     22
                          ... 
open end question           11
want kno

## the same process with tfidf vectorizer 

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
tfidf_vectorizer = TfidfVectorizer(analyzer='word',
                             ngram_range = (3, 3),
                             min_df = 10,                      # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

In [31]:
data_tfidf_vectorized = tfidf_vectorizer.fit_transform(data_lemmatized)

In [32]:
# Materialize the sparse data
data_tfidf_dense = data_tfidf_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_tfidf_dense > 0).sum()/data_tfidf_dense.size)*100, "%")

Sparsicity:  2.902957692229723 %


In [42]:
threegrams = pd.DataFrame(data_tfidf_dense,
                 columns=tfidf_vectorizer.get_feature_names())

In [48]:
threegramscombined = pd.concat([interviews, threegrams], axis = 1)

## specifically looking for two grams 

In [34]:
tfidf_vectorizer_two = TfidfVectorizer(analyzer='word',
                             ngram_range = (2, 2),
                             min_df = 10,                      # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

In [35]:
data_tfidf_vectorized_two = tfidf_vectorizer_two.fit_transform(data_lemmatized)

In [36]:
# Materialize the sparse data
data_tfidf_dense_two = data_tfidf_vectorized_two.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_tfidf_dense_two > 0).sum()/data_tfidf_dense_two.size)*100, "%")

Sparsicity:  4.326159121788859 %


In [43]:
twograms = pd.DataFrame(data_tfidf_dense_two,
                 columns=tfidf_vectorizer_two.get_feature_names())

In [50]:
combined = pd.concat([threegramscombined, twograms], axis = 1)

## now only words

In [38]:
tfidf_vectorizer_one = TfidfVectorizer(analyzer='word',
                             ngram_range = (1, 1),
                             min_df = 10,                      # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

In [39]:
data_tfidf_vectorized_one = tfidf_vectorizer_one.fit_transform(data_lemmatized)

In [40]:
# Materialize the sparse data
data_tfidf_dense_one = data_tfidf_vectorized_one.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_tfidf_dense_one > 0).sum()/data_tfidf_dense_one.size)*100, "%")

Sparsicity:  11.48197394322927 %


In [44]:
words = pd.DataFrame(data_tfidf_dense_one,
                 columns=tfidf_vectorizer_one.get_feature_names())

In [51]:
combined = pd.concat([combined, words], axis = 1)

In [53]:
combined.head(1)

,filename,text,Name,Gender,Organization,Current Age,NativeSpeaker,Traded,MBA,Sector_Grouped,...,yell,yellow,yesterday,yield,york,young,youth,youtube,zone,zoom
0,AaronBellCombined.txt,Very well. Thanks for having me.\nYeah. So a little bit about adroll. First of all we were founded in 2007 uh with a...,AaronBell,M,AdRoll,42,1,0,0,2,...,0.0,0.0,0.0,0.0,0.013131,0.0,0.0,0.0,0.017125,0.0


In [59]:
combined.tail(5)

,filename,text,Name,Gender,Organization,Current Age,NativeSpeaker,Traded,MBA,Sector_Grouped,...,yell,yellow,yesterday,yield,york,young,youth,youtube,zone,zoom
517,WendyLeaCombined.txt,I am running a startup catalyst publicprivate partnership that exists to build a sustainable tech-based economy fo...,WendyLea,F,Get Satisfaction Inc,65,1,0,0,2,...,0.0,0.000000,0.000000,0.000000,0.014389,0.010037,0.0,0.0,0.000000,0.0
518,WillWrightCombined.txt,Basically its a game about life in the broadest possible sense. You start as a single-cell microbe at the beginning ...,WillWright,M,Maxis,59,1,0,0,3,...,0.0,0.047405,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
519,WilliamDGreenCombined.txt,Were a very client-focused company. If you go back in our history we used to provide services but now we provide bus...,WilliamDGreen,M,Accenture LLP,66,1,0,1,1,...,0.0,0.000000,0.000000,0.020139,0.029417,0.000000,0.0,0.0,0.000000,0.0
520,YorgenEdholmCombined.txt,Founded in 1999 Palo Alto California-based Accellion has annual revenue closer to 50 million than 100 million and is...,YorgenEdholm,M,Accellion Inc,64,0,0,1,3,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
521,YuchunLeeCombined.txt,Allego is now in its sixth year and is cash flow positive so it was not about needing the funds. In fact I have turn...,YuchunLee,M,Allego,54,1,0,1,3,...,0.0,0.000000,0.026323,0.000000,0.052280,0.048623,0.0,0.0,0.022726,0.0


## now LIWC categories 

In [45]:
LIWC = pd.read_excel('C:/Users/2918261/Dropbox/Corner Office Interviews/ComparisonCorpusText/LIWC_Combined_Full.xlsx')

In [46]:
LIWC.head()

,Name,Gender,Organization,Fortune,Position,Current Age,Higher Education Info,Birth Country,Stock Market Info,BachelorsMin,...,Exclam,Dash,Quote,Apostro,Parenth,OtherP,Female,Sector_Grouped_Dic,Femininity,Masculinity
0,AaronBell,M,AdRoll,1,CEO,42,BS,USA,NaN,1,...,0,0.14,0.0,0.0,0,0.0,0,Communications,64.47,38.83
1,AaronLevie,M,Box Inc,1,CEO,33,College Dropout,USA,NYSE,0,...,0,0.33,0.0,0.0,0,0.0,0,Technology,63.35,37.73
2,AbbeRaven,F,A&E Television Networks LLC,1,Acting Chairman,66,BA MA Honorary Doctorate,USA,NaN,1,...,0,0.39,0.0,0.0,0,0.0,1,Communications,62.63,38.94
3,AbeAnkumah,M,Nyansa,0,CEO,41,BS MBA,Ghana,NaN,1,...,0,0.98,0.0,0.0,0,0.0,0,Technology,65.51,38.98
4,AdamNash,M,Wealthfront,0,Former CEO and President,44,BA MA MBA,USA,NaN,1,...,0,0.35,0.0,0.0,0,0.0,0,Other,62.76,40.87


## make all the column names upper case

In [54]:
LIWC.columns = map(str.upper, LIWC.columns)

In [56]:
list(LIWC.columns)

['NAME ',
 'GENDER',
 'ORGANIZATION',
 'FORTUNE',
 'POSITION ',
 'CURRENT AGE',
 'HIGHER EDUCATION INFO',
 'BIRTH COUNTRY',
 'STOCK MARKET INFO',
 'BACHELORSMIN',
 'NATIVESPEAKER',
 'TRADED',
 'MBA',
 'TITLELENGTH',
 'PODCAST',
 'PODCAST_DATE',
 'PODCAST_LINK',
 'PODCAST_DETAILS',
 'INTERVIEW_DATE',
 'INTERVIEW',
 'INTERVIEW_LINK',
 'INTERVIEW_DETAILS',
 'CAPTION_DATE',
 'CAPTION',
 'CAPTION_LINK',
 'CAPTION_DETAILS',
 'ESSAY_DATE',
 'ESSAY_TITLE',
 'ESSAY',
 'ESSAY_LINK ',
 'OTHER SOURCES_DATE',
 'OTHER SOURCES ',
 'OTHER SOURCES_LINK',
 'OTHER SOURCES_DETAILS',
 'SECTOR_GROUPED',
 'WC',
 'ANALYTIC',
 'CLOUT',
 'AUTHENTIC',
 'TONE',
 'WPS',
 'SIXLTR',
 'DIC',
 'FUNCTION',
 'PRONOUN',
 'PPRON',
 'I',
 'WE',
 'YOU',
 'SHEHE',
 'THEY',
 'IPRON',
 'ARTICLE',
 'PREP',
 'AUXVERB',
 'ADVERB',
 'CONJ',
 'NEGATE',
 'VERB',
 'ADJ',
 'COMPARE',
 'INTERROG',
 'NUMBER',
 'QUANT',
 'AFFECT',
 'POSEMO',
 'NEGEMO',
 'ANX',
 'ANGER',
 'SAD',
 'SOCIAL',
 'FAMILY',
 'FRIEND',
 'FEMALE',
 'MALE',
 'COGPR

## drop the some of the columns to avoid duplicity

In [60]:
LIWC = LIWC.drop(['GENDER','FORTUNE',
 'POSITION ',
 'CURRENT AGE',
 'HIGHER EDUCATION INFO',
 'BIRTH COUNTRY',
 'STOCK MARKET INFO',
 'BACHELORSMIN',
 'NATIVESPEAKER',
 'TRADED',
 'MBA',
 'TITLELENGTH',
 'PODCAST',
 'PODCAST_DATE',
 'PODCAST_LINK',
 'PODCAST_DETAILS',
 'INTERVIEW_DATE',
 'INTERVIEW',
 'INTERVIEW_LINK',
 'INTERVIEW_DETAILS',
 'CAPTION_DATE',
 'CAPTION',
 'CAPTION_LINK',
 'CAPTION_DETAILS',
 'ESSAY_DATE',
 'ESSAY_TITLE',
 'ESSAY',
 'ESSAY_LINK ',
 'OTHER SOURCES_DATE',
 'OTHER SOURCES ',
 'OTHER SOURCES_LINK',
 'OTHER SOURCES_DETAILS',
 'SECTOR_GROUPED',
 'WC',
 'ANALYTIC',
 'CLOUT',
 'AUTHENTIC',
 'TONE',
 'WPS',
 'SIXLTR',
 'DIC',
 'NETSPEAK',
 'ASSENT',
 'NONFLU',
 'FILLER',
 'ALLPUNC',
 'PERIOD',
 'COMMA',
 'COLON',
 'SEMIC',
 'QMARK',
 'EXCLAM',
 'DASH',
 'QUOTE',
 'APOSTRO',
 'PARENTH',
 'OTHERP',
 'FEMALE',
 'SECTOR_GROUPED_DIC',
 'FEMININITY',
 'MASCULINITY'], axis = 1).copy()

In [62]:
combined = pd.concat([combined, LIWC], axis = 1)

In [63]:
combined[['Name ', 'filename', 'NAME ']]

,Name,filename,NAME
0,AaronBell,AaronBellCombined.txt,AaronBell
1,AaronLevie,AaronLevieCombined.txt,AaronLevie
2,AbbeRaven,AbbeRavenCombined.txt,AbbeRaven
3,AbeAnkumah,AbeAnkumahCombined.txt,AbeAnkumah
4,AdamNash,AdamNashCombined.txt,AdamNash
5,AdiTatarko,AdiTatarkoCombined.txt,AdiTatarko
6,AileenLee,AileenLeeCombined.txt,AileenLee
7,AlanDabbiere,AlanDabbiereCombined.txt,AlanDabbiere
8,AlanRMulally,AlanRMulallyCombined.txt,AlanRMulally
9,AlanTrefler,AlanTreflerCombined.txt,AlanTrefler


In [64]:
list(combined.columns)

['filename',
 'text',
 'Name ',
 'Gender',
 'Organization',
 'Current Age',
 'NativeSpeaker',
 'Traded',
 'MBA',
 'Sector_Grouped',
 'Female',
 'Sector_Grouped_Dic',
 'Femininity',
 'Masculinity',
 'ago year ago',
 'ask lot question',
 'ask people tell',
 'ask right question',
 'ask tell time',
 'bad thing happen',
 'big mistake make',
 'build great company',
 'challenge status quo',
 'chief operating officer',
 'come say know',
 'come work day',
 'company year ago',
 'couple year ago',
 'create environment people',
 'decision make decision',
 'different point view',
 'good thing happen',
 'graduate high school',
 'grow small town',
 'harvard business school',
 'high school college',
 'high school student',
 'high school work',
 'hire good people',
 'hire people really',
 'important make sure',
 'important thing learn',
 'job make sure',
 'just little bit',
 'just make decision',
 'just make sure',
 'just say know',
 'just want know',
 'just want say',
 'know just know',
 'know know kn

In [66]:
combined = combined.drop(['NAME '], axis = 1).copy()

In [72]:
combined = combined.reset_index(drop=True).copy()

In [79]:
combined = combined.drop(['text', 'filename'], axis = 1).copy()

In [75]:
import json

In [80]:
combined.to_json('tokensmeta.json')